# Finding defintions with **S**
If we want to define term X, how can we do so based on information extracted from a word embedding? I am proposing and will explore the use of a simple relation of the two common vector descriptors - angle and magnitude. When defining a word, I will assign each word in our vocabulary some kind of score, **S**, that will help us define X with some pruning.
```
S(X, y) = cosine_similarity(X, y) * |y|
```
This is based on the idea that words with a larger magnitude appear more frequently in the test set, and therefore are likely to be more simple (or at least common). By using each to scale the other, we project both important pieces of information into the same space, and can begin to construct a definition.

How do we avoid orthographic influences? After generating S(X, Y), I will prune out any remaining 'descriptors' that share the same 3-letter sequence to start the word. Hopefully this will reduce "man" being defined using "manner" or "manuscript", or something similar. Once the list is pruned, I will chose the two highest scoring terms in S(X, Y) that do not contain orthographic similarity, and output them as our "definition". This will complete the statement "X is a(n) y1 y2" (ex. "bachelor is an unmarried man").

In [ ]:
import os
import torch
import openai
import numpy as np

import sentencepiece    # necessary for proper t5 init.
from transformers import T5Tokenizer, T5EncoderModel, GPT2Tokenizer, OPTModel

from sklearn.metrics.pairwise import cosine_similarity

# api key set in conda env.
openai.api_key = os.getenv('OPENAI_API_KEY')

## Loading Models
You know the drill.

### Vocab

In [ ]:
vocab = []
with open('expanded_vocab.txt', 'r') as f:
    for line in f:
        vocab.append(line.strip())

len(vocab)

### OPT-1.3B

In [ ]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('facebook/opt-1.3b', cache_dir='/scratch/mbarlow6/cache')
model_opt_raw = OPTModel.from_pretrained('facebook/opt-1.3b', cache_dir='/scratch/mbarlow6/cache')

In [ ]:
opt_embeds = []
with open(u'./opt/1_3B.txt', 'r') as f:
    for line in f:
        opt_embeds.append([float(x) for x in line.strip().split()])
model_opt = dict(zip(vocab, opt_embeds))

In [ ]:
def opt_embed(text, tokenizer=gpt2_tokenizer, model=model_opt_raw, debug=False):
    inputs = tokenizer(text, return_tensors='pt')
    if debug:
        print('Tokens Requested:')
        print(tokenizer.batch_decode(inputs.input_ids[0]))
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = torch.squeeze(outputs.last_hidden_state, dim=0)
    return np.array(torch.mean(embeddings[1:], dim=0))

### T5

In [ ]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-large', cache_dir='/scratch/mbarlow6/cache')
model_t5_raw = T5EncoderModel.from_pretrained('t5-large', cache_dir='/scratch/mbarlow6/cache')

In [ ]:
t5_embeds = []
with open('./t5/t5large.txt', 'r') as f:
    for line in f:
        t5_embeds.append([float(x) for x in line.strip().split()])
model_t5 = dict(zip(vocab, t5_embeds))

In [ ]:
def t5_embed(text, tokenizer=t5_tokenizer, model=model_t5_raw, debug=False):
    inputs = tokenizer(text, return_tensors='pt')
    if debug:
        print('Tokens Requested:')
        print(tokenizer.batch_decode(inputs.input_ids[0]))
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = torch.squeeze(outputs.last_hidden_state, dim=0)
    return np.array(torch.mean(embeddings, dim=0))

## Helpers
Positive for now.

In [ ]:
def positive(words, model='opt'):
    """
    Args:
        words: iterable
        model: 'opt', or 't5'
    Returns:
        Positive (summed vectors) of word embeddings of a given list of words from the specified model. Defaults to GPT-3.
    """
    if isinstance(words, str):
        print(f"You requested the positive of the string \"{words}\". Did you mean [\"{words}\"]?")

    out = 0
    for token in words:
        # convert token to string
        word = str(token)
        # do model check - least intensive operation to repeat
        if model.lower() == 'opt':
            if word in model_opt:
                ex = model_opt[word]
            else:
                # squeeze!
                ex = opt_embed(word)
                model_opt[word] = ex
        elif model.lower() == 't5':
            if word in model_t5:
                ex = model_t5[word]
            else:
                ex = t5_embed(word)
                model_t5[word] = ex
        else:
            raise ValueError('Please provide either opt or t5 as a model choice.')

        # construct positive
        if isinstance(out, int):
            out = np.array(ex).reshape(1, -1)
        else:
            out += np.array(ex).reshape(1, -1)
            
    return out if not isinstance(out, int) else np.array([])

In [ ]:
def bigs_define(phrase, n=2, model='opt'):
    """
    Args:
        phrase: iterable or string  -> Word(s) to be defined using big S.
        n: int                      -> Number of terms in definition. Defaults to 2.
        model: 'opt' or 't5'        -> Model to use. Defaults to OPT.
    Returns:
        Defintion of phrase using n terms derived from chosen model's weights.
    """
    # normalize phrase to list
    if isinstance(phrase, str):
        phrase = [phrase]
    phrase = positive(phrase, model=model)

    # initialize S
    S = []

    # score vocab
    for term in vocab:
        b = positive([term], model=model)
        
        mag = np.sqrt(b.dot(b))
        s = cosine_similarity(phrase, b) * mag

        S.append((term, s))

    # sort S
    S.sort(key=lambda x: x[1])

    # get answer
    ans = []
    while len(ans) < n and len(S) > 0:
        # check S[0][0] for orthographic similarity
        flag = False
        for word in phrase:
            if word[:3] == S[-1][0][:3]:
                flag = True
                break
        
        if flag:
            S.pop(-1)
            continue

        ans.append(S.pop(-1))

    return ans

## Tests
Now we check if this is viable!

In [ ]:
bigs_define(['puppy'])

In [ ]:
bigs_define(['bachelor'])

In [ ]:
bigs_define(['wife'])

In [ ]:
bigs_define(['husband'])

In [ ]:
bigs_define(['king'])

In [ ]:
bigs_define(['duckling'])

In [ ]:
bigs_define(['kitten'])

In [ ]:
bigs_define(['knowledge'])